# Focused Learning: Synthetic Data Generation with GPT Translation

## Learning Objective
Master the techniques for generating high-quality synthetic programming language datasets through translation, as described in Section III.D (KExercises dataset) of the paper.

## Paper Reference
- **Section**: III.D - KExercises: Kotlin instructions dataset
- **Key Result**: Synthetic dataset (KExercises) achieved best performance (55.28% pass rate)
- **Method**: GPT-3.5-turbo translation from Python exercises to Kotlin

## 1. The Challenge: Low-Resource Languages

### 1.1 Why Synthetic Data?

From the paper: "Kotlin could be considered a low-resource language due to the scarcity of publicly available data and the limited opportunities for improvement using data collected from open-source projects."

Key challenges:
- Limited public Kotlin repositories
- Most Kotlin code is in private/enterprise repositories
- Need diverse examples covering various programming concepts

In [ ]:
# Install required packages
!pip install langchain langchain-openai pandas numpy matplotlib seaborn

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from typing import List, Dict, Tuple, Optional
from dataclasses import dataclass
import json
import re
from collections import Counter

# LangChain imports for translation pipeline
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema import SystemMessage, HumanMessage
from langchain.chains import LLMChain

# Set visualization style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (10, 6)

## 2. Understanding the Source: Python Exercises Dataset

### 2.1 Characteristics of Good Exercise Data

According to Gunasekar et al. (referenced in the paper), synthetic data should:
- Cover broad spectrum of coding concepts
- Vary in difficulty and complexity
- Include diverse coding styles
- Have clear problem statements and solutions

In [ ]:
@dataclass
class CodeExercise:
    """Represents a programming exercise for translation"""
    id: str
    description: str
    python_code: str
    difficulty: str  # easy, medium, hard
    concepts: List[str]  # e.g., ['loops', 'arrays', 'sorting']
    test_cases: Optional[List[Dict]] = None

# Example exercises covering different concepts
sample_exercises = [
    CodeExercise(
        id="ex001",
        description="""Write a function that finds the maximum difference between any two elements in an array,
where the larger element appears after the smaller element.""",
        python_code="""def max_profit(prices: List[int]) -> int:
    '''Find maximum profit from buying and selling stock.
    
    Args:
        prices: List of daily stock prices
        
    Returns:
        Maximum profit possible (0 if no profit)
    '''
    if not prices:
        return 0
    
    min_price = prices[0]
    max_profit = 0
    
    for price in prices[1:]:
        if price < min_price:
            min_price = price
        else:
            max_profit = max(max_profit, price - min_price)
    
    return max_profit""",
        difficulty="medium",
        concepts=["arrays", "dynamic_programming", "optimization"]
    ),
    
    CodeExercise(
        id="ex002",
        description="""Implement a function that checks if a string is a valid palindrome,
considering only alphanumeric characters and ignoring case.""",
        python_code="""def is_palindrome(s: str) -> bool:
    '''Check if string is a palindrome (alphanumeric only).
    
    Args:
        s: Input string
        
    Returns:
        True if palindrome, False otherwise
    '''
    # Filter alphanumeric and convert to lowercase
    cleaned = ''.join(c.lower() for c in s if c.isalnum())
    
    # Check palindrome
    return cleaned == cleaned[::-1]""",
        difficulty="easy",
        concepts=["strings", "two_pointers"]
    ),
    
    CodeExercise(
        id="ex003",
        description="""Design a class that implements a Least Recently Used (LRU) cache
with get and put operations in O(1) time complexity.""",
        python_code="""from collections import OrderedDict

class LRUCache:
    '''Least Recently Used cache implementation.
    
    Supports get and put operations in O(1) time.
    '''
    
    def __init__(self, capacity: int):
        self.capacity = capacity
        self.cache = OrderedDict()
    
    def get(self, key: int) -> int:
        '''Get value for key, -1 if not exists.'''
        if key not in self.cache:
            return -1
        # Move to end (most recent)
        self.cache.move_to_end(key)
        return self.cache[key]
    
    def put(self, key: int, value: int) -> None:
        '''Put key-value pair in cache.'''
        if key in self.cache:
            self.cache.move_to_end(key)
        self.cache[key] = value
        if len(self.cache) > self.capacity:
            # Remove least recently used
            self.cache.popitem(last=False)""",
        difficulty="hard",
        concepts=["data_structures", "design", "optimization"]
    )
]

# Analyze exercise distribution
def analyze_exercise_distribution(exercises: List[CodeExercise]):
    """Analyze the distribution of exercises by difficulty and concepts"""
    difficulties = Counter(ex.difficulty for ex in exercises)
    all_concepts = []
    for ex in exercises:
        all_concepts.extend(ex.concepts)
    concept_counts = Counter(all_concepts)
    
    return difficulties, concept_counts

difficulties, concepts = analyze_exercise_distribution(sample_exercises)
print("Exercise Distribution:")
print(f"Difficulties: {dict(difficulties)}")
print(f"Top concepts: {dict(list(concepts.most_common(5)))}")

## 3. The Translation Pipeline

### 3.1 Translation Prompt from the Paper

The paper provides the exact prompt used (Figure 2):
```
System: You are a helpful assistant.
User: Rewrite to Kotlin (do not forget about docstring):

PYTHON_CODE
```

In [ ]:
class KotlinTranslator:
    """Translates Python exercises to Kotlin following the paper's approach"""
    
    def __init__(self, use_mock=True):
        """
        Args:
            use_mock: If True, use mock translations for demonstration
        """
        self.use_mock = use_mock
        
        if not use_mock:
            self.llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
        
        # Exact prompt from the paper
        self.prompt = ChatPromptTemplate.from_messages([
            SystemMessage(content="You are a helpful assistant."),
            HumanMessage(content="""Rewrite to Kotlin (do not forget about docstring):

{python_code}""")
        ])
        
        if not use_mock:
            self.chain = LLMChain(llm=self.llm, prompt=self.prompt)
    
    def translate(self, exercise: CodeExercise) -> Dict[str, str]:
        """Translate a Python exercise to Kotlin"""
        if self.use_mock:
            return self._mock_translate(exercise)
        else:
            kotlin_code = self.chain.run(python_code=exercise.python_code)
            return {
                'kotlin_code': kotlin_code,
                'status': 'success'
            }
    
    def _mock_translate(self, exercise: CodeExercise) -> Dict[str, str]:
        """Mock translation for demonstration"""
        # Create realistic Kotlin translations
        translations = {
            "ex001": """fun maxProfit(prices: IntArray): Int {
    /**
     * Find maximum profit from buying and selling stock.
     * 
     * @param prices List of daily stock prices
     * @return Maximum profit possible (0 if no profit)
     */
    if (prices.isEmpty()) {
        return 0
    }
    
    var minPrice = prices[0]
    var maxProfit = 0
    
    for (i in 1 until prices.size) {
        val price = prices[i]
        if (price < minPrice) {
            minPrice = price
        } else {
            maxProfit = maxOf(maxProfit, price - minPrice)
        }
    }
    
    return maxProfit
}""",
            
            "ex002": """fun isPalindrome(s: String): Boolean {
    /**
     * Check if string is a palindrome (alphanumeric only).
     * 
     * @param s Input string
     * @return True if palindrome, false otherwise
     */
    // Filter alphanumeric and convert to lowercase
    val cleaned = s.filter { it.isLetterOrDigit() }.lowercase()
    
    // Check palindrome
    return cleaned == cleaned.reversed()
}""",
            
            "ex003": """class LRUCache(private val capacity: Int) {
    /**
     * Least Recently Used cache implementation.
     * 
     * Supports get and put operations in O(1) time.
     */
    
    private val cache = LinkedHashMap<Int, Int>(capacity + 1, 0.75f, true)
    
    fun get(key: Int): Int {
        /**
         * Get value for key, -1 if not exists.
         */
        return cache[key] ?: -1
    }
    
    fun put(key: Int, value: Int) {
        /**
         * Put key-value pair in cache.
         */
        cache[key] = value
        if (cache.size > capacity) {
            // Remove least recently used (first entry)
            val iterator = cache.keys.iterator()
            iterator.next()
            iterator.remove()
        }
    }
}"""
        }
        
        return {
            'kotlin_code': translations.get(exercise.id, "// Translation not available"),
            'status': 'success'
        }
    
    def batch_translate(self, exercises: List[CodeExercise], 
                       monitor_quality: bool = True) -> List[Dict]:
        """Translate multiple exercises with quality monitoring"""
        results = []
        
        for exercise in exercises:
            result = self.translate(exercise)
            
            if monitor_quality:
                # Add quality metrics
                result['quality_score'] = self._assess_translation_quality(
                    exercise.python_code, 
                    result['kotlin_code']
                )
            
            result['exercise_id'] = exercise.id
            results.append(result)
        
        return results
    
    def _assess_translation_quality(self, python_code: str, kotlin_code: str) -> float:
        """Simple heuristic for translation quality"""
        score = 1.0
        
        # Check for proper Kotlin documentation
        if '/**' not in kotlin_code:
            score -= 0.2
        
        # Check for Kotlin idioms
        kotlin_idioms = ['fun ', 'val ', 'var ', 'when ', 'lateinit', 'companion object']
        idiom_count = sum(1 for idiom in kotlin_idioms if idiom in kotlin_code)
        score = min(score + idiom_count * 0.05, 1.0)
        
        # Check structure preservation
        python_lines = len(python_code.split('\n'))
        kotlin_lines = len(kotlin_code.split('\n'))
        if abs(python_lines - kotlin_lines) / python_lines > 0.5:
            score -= 0.1
        
        return max(score, 0.0)

## 4. Translation in Action

Let's demonstrate the translation process and analyze the results.

In [ ]:
# Create translator and translate exercises
translator = KotlinTranslator(use_mock=True)
translation_results = translator.batch_translate(sample_exercises)

# Display translations side by side
for i, (exercise, result) in enumerate(zip(sample_exercises, translation_results)):
    print(f"\n{'='*80}")
    print(f"Exercise {i+1}: {exercise.description[:60]}...")
    print(f"Difficulty: {exercise.difficulty} | Concepts: {', '.join(exercise.concepts)}")
    print(f"Quality Score: {result['quality_score']:.2f}")
    print(f"\n{'Python':-^40} | {'Kotlin':-^40}")
    print("-" * 81)
    
    # Split and display code side by side
    python_lines = exercise.python_code.split('\n')
    kotlin_lines = result['kotlin_code'].split('\n')
    
    max_lines = max(len(python_lines), len(kotlin_lines))
    for j in range(min(10, max_lines)):  # Show first 10 lines
        py_line = python_lines[j] if j < len(python_lines) else ""
        kt_line = kotlin_lines[j] if j < len(kotlin_lines) else ""
        print(f"{py_line[:40]:<40} | {kt_line[:40]:<40}")
    
    if max_lines > 10:
        print(f"{'...':<40} | {'...':<40}")

## 5. Quality Control and Validation

### 5.1 Paper's Approach

From Section III.D: "We iteratively translated segments of data and monitored the downstream Kotlin generation quality during validation. Additionally, after the translation, we manually reviewed a sample of the data to ensure the accuracy of the translations."

In [ ]:
class TranslationValidator:
    """Validates translated Kotlin code quality"""
    
    def __init__(self):
        self.validation_rules = [
            self._check_syntax_markers,
            self._check_documentation,
            self._check_kotlin_idioms,
            self._check_type_safety,
            self._check_null_safety
        ]
    
    def validate(self, kotlin_code: str) -> Dict[str, any]:
        """Run all validation checks on Kotlin code"""
        results = {
            'valid': True,
            'issues': [],
            'warnings': [],
            'score': 1.0
        }
        
        for rule in self.validation_rules:
            rule_result = rule(kotlin_code)
            if not rule_result['passed']:
                results['score'] -= rule_result['penalty']
                if rule_result['severity'] == 'error':
                    results['valid'] = False
                    results['issues'].append(rule_result['message'])
                else:
                    results['warnings'].append(rule_result['message'])
        
        results['score'] = max(0, results['score'])
        return results
    
    def _check_syntax_markers(self, code: str) -> Dict:
        """Check for basic Kotlin syntax"""
        if 'def ' in code or '__init__' in code:
            return {
                'passed': False,
                'severity': 'error',
                'penalty': 0.5,
                'message': 'Python syntax found in Kotlin code'
            }
        return {'passed': True, 'penalty': 0}
    
    def _check_documentation(self, code: str) -> Dict:
        """Check for proper Kotlin documentation"""
        if '/**' not in code and 'fun ' in code:
            return {
                'passed': False,
                'severity': 'warning',
                'penalty': 0.1,
                'message': 'Missing KDoc documentation'
            }
        return {'passed': True, 'penalty': 0}
    
    def _check_kotlin_idioms(self, code: str) -> Dict:
        """Check for Kotlin-specific idioms"""
        # Check for proper use of val/var
        if re.search(r'\blet\s+\w+\s*=', code):  # JavaScript style
            return {
                'passed': False,
                'severity': 'error',
                'penalty': 0.3,
                'message': 'Non-Kotlin variable declaration found'
            }
        return {'passed': True, 'penalty': 0}
    
    def _check_type_safety(self, code: str) -> Dict:
        """Check for type annotations"""
        function_matches = re.findall(r'fun\s+\w+\s*\([^)]*\)', code)
        for match in function_matches:
            if ':' not in match and '()' not in match:  # Parameters should have types
                return {
                    'passed': False,
                    'severity': 'warning',
                    'penalty': 0.2,
                    'message': 'Missing type annotations in function parameters'
                }
        return {'passed': True, 'penalty': 0}
    
    def _check_null_safety(self, code: str) -> Dict:
        """Check for proper null safety handling"""
        # Simple heuristic: check for !! usage
        if '!!' in code and 'TODO' not in code:
            return {
                'passed': False,
                'severity': 'warning',
                'penalty': 0.05,
                'message': 'Using !! operator - consider safer null handling'
            }
        return {'passed': True, 'penalty': 0}

# Validate translations
validator = TranslationValidator()

print("Translation Validation Results:")
print("=" * 60)

for result in translation_results:
    validation = validator.validate(result['kotlin_code'])
    print(f"\nExercise: {result['exercise_id']}")
    print(f"Valid: {'✅' if validation['valid'] else '❌'}")
    print(f"Quality Score: {validation['score']:.2f}")
    
    if validation['issues']:
        print("Issues:")
        for issue in validation['issues']:
            print(f"  - {issue}")
    
    if validation['warnings']:
        print("Warnings:")
        for warning in validation['warnings']:
            print(f"  - {warning}")

## 6. Dataset Characteristics and Impact

### 6.1 Final Dataset Statistics

From the paper:
- 15,000 Kotlin tasks
- ~3.5 million tokens
- 335,000 lines of code

In [ ]:
# Simulate dataset statistics
def analyze_kexercises_dataset():
    """Analyze the characteristics of the KExercises dataset"""
    
    # Statistics from the paper
    dataset_stats = {
        'total_exercises': 15000,
        'total_tokens': 3_500_000,
        'total_lines': 335_000,
        'avg_tokens_per_exercise': 3_500_000 / 15000,
        'avg_lines_per_exercise': 335_000 / 15000
    }
    
    # Simulate distribution of exercise characteristics
    np.random.seed(42)
    
    # Difficulty distribution (assumed)
    difficulties = np.random.choice(
        ['easy', 'medium', 'hard'], 
        size=1000, 
        p=[0.3, 0.5, 0.2]
    )
    
    # Lines per exercise (log-normal distribution)
    lines_per_exercise = np.random.lognormal(
        mean=np.log(dataset_stats['avg_lines_per_exercise']), 
        sigma=0.5, 
        size=1000
    )
    
    # Concepts coverage (simulated)
    all_concepts = [
        'functions', 'classes', 'loops', 'conditionals', 'arrays',
        'strings', 'data_structures', 'algorithms', 'io', 'error_handling',
        'generics', 'coroutines', 'lambdas', 'collections', 'null_safety'
    ]
    
    concept_coverage = {concept: np.random.randint(500, 3000) for concept in all_concepts}
    
    return dataset_stats, difficulties, lines_per_exercise, concept_coverage

stats, difficulties, lines, concepts = analyze_kexercises_dataset()

# Visualize dataset characteristics
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# Plot 1: Dataset size comparison
datasets = ['KStack', 'KStack-clean', 'KExercises']
tokens = [3.1e9, 22e6, 3.5e6]
lines = [293e6, 2e6, 335e3]

ax1 = axes[0, 0]
x = np.arange(len(datasets))
width = 0.35
ax1.bar(x - width/2, np.log10(tokens), width, label='Tokens (log10)')
ax1.bar(x + width/2, np.log10(lines), width, label='Lines (log10)')
ax1.set_xlabel('Dataset')
ax1.set_ylabel('Size (log10 scale)')
ax1.set_title('Dataset Size Comparison')
ax1.set_xticks(x)
ax1.set_xticklabels(datasets)
ax1.legend()
ax1.grid(True, alpha=0.3)

# Plot 2: Difficulty distribution
ax2 = axes[0, 1]
diff_counts = pd.Series(difficulties).value_counts()
ax2.pie(diff_counts.values, labels=diff_counts.index, autopct='%1.1f%%', 
        colors=['lightgreen', 'gold', 'lightcoral'])
ax2.set_title('Exercise Difficulty Distribution')

# Plot 3: Lines per exercise distribution
ax3 = axes[1, 0]
ax3.hist(lines, bins=50, color='skyblue', alpha=0.7, edgecolor='black')
ax3.axvline(stats['avg_lines_per_exercise'], color='red', linestyle='--', 
            label=f"Average: {stats['avg_lines_per_exercise']:.1f}")
ax3.set_xlabel('Lines per Exercise')
ax3.set_ylabel('Frequency')
ax3.set_title('Exercise Length Distribution')
ax3.legend()
ax3.grid(True, alpha=0.3)

# Plot 4: Concept coverage
ax4 = axes[1, 1]
top_concepts = sorted(concepts.items(), key=lambda x: x[1], reverse=True)[:10]
concept_names = [c[0] for c in top_concepts]
concept_counts = [c[1] for c in top_concepts]
ax4.barh(concept_names, concept_counts, color='purple', alpha=0.7)
ax4.set_xlabel('Number of Exercises')
ax4.set_title('Top 10 Programming Concepts Coverage')
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Print key statistics
print("\nKExercises Dataset Summary:")
print("=" * 50)
for key, value in stats.items():
    print(f"{key.replace('_', ' ').title():<30}: {value:,.1f}")

## 7. Performance Impact Analysis

### 7.1 Why KExercises Outperformed Other Datasets

From Table II in the paper, KExercises achieved the best results across all models tested.

In [ ]:
# Performance results from Table II
performance_results = pd.DataFrame({
    'Model': ['CodeLlama-7B', 'CodeLlama-7B', 'CodeLlama-7B', 'CodeLlama-7B',
              'Deepseek-7B', 'Deepseek-7B', 'Deepseek-1.3B', 'Deepseek-1.3B', 'Deepseek-1.3B'],
    'Dataset': ['Base', 'KStack', 'KStack-clean', 'KExercises',
                'Base', 'KExercises', 'Base', 'KStack', 'KExercises'],
    'Pass_Rate': [26.09, 29.19, 37.89, 42.24, 40.99, 55.28, 26.71, 27.95, 36.65],
    'Syntax_Error_Rate': [22.98, 22.98, 18.64, 19.25, 21.12, 15.53, 19.26, 19.88, 18.63],
    'Completion_Rate': [0.388, 0.396, 0.403, 0.344, 0.403, 0.411, 0.403, 0.404, 0.388]
})

# Calculate improvements
def calculate_improvements(df):
    improvements = []
    for model in ['CodeLlama-7B', 'Deepseek-7B', 'Deepseek-1.3B']:
        model_data = df[df['Model'] == model]
        base_pass = model_data[model_data['Dataset'] == 'Base']['Pass_Rate'].values[0]
        kexer_data = model_data[model_data['Dataset'] == 'KExercises']
        if len(kexer_data) > 0:
            kexer_pass = kexer_data['Pass_Rate'].values[0]
            improvement = kexer_pass - base_pass
            improvements.append({
                'Model': model,
                'Base_Pass_Rate': base_pass,
                'KExercises_Pass_Rate': kexer_pass,
                'Improvement': improvement,
                'Improvement_Pct': (improvement / base_pass) * 100
            })
    return pd.DataFrame(improvements)

improvements_df = calculate_improvements(performance_results)

# Visualize improvements
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# Bar chart of absolute improvements
x = np.arange(len(improvements_df))
bars = ax1.bar(x, improvements_df['Improvement'], color=['#1f77b4', '#ff7f0e', '#2ca02c'])
ax1.set_xlabel('Model')
ax1.set_ylabel('Pass Rate Improvement (pp)')
ax1.set_title('Absolute Improvement with KExercises')
ax1.set_xticks(x)
ax1.set_xticklabels(improvements_df['Model'])
ax1.grid(True, alpha=0.3, axis='y')

# Add value labels on bars
for bar, imp in zip(bars, improvements_df['Improvement']):
    height = bar.get_height()
    ax1.text(bar.get_x() + bar.get_width()/2., height + 0.5,
             f'+{imp:.1f}pp', ha='center', va='bottom')

# Comparison across all datasets for one model
codellama_data = performance_results[performance_results['Model'] == 'CodeLlama-7B']
datasets = codellama_data['Dataset'].values
pass_rates = codellama_data['Pass_Rate'].values

colors = ['lightcoral', 'lightsalmon', 'lightblue', 'lightgreen']
bars = ax2.bar(range(len(datasets)), pass_rates, color=colors)
ax2.set_xlabel('Dataset')
ax2.set_ylabel('Pass Rate (%)')
ax2.set_title('CodeLlama-7B Performance Across Datasets')
ax2.set_xticks(range(len(datasets)))
ax2.set_xticklabels(datasets)
ax2.grid(True, alpha=0.3, axis='y')

# Add improvement annotations
for i in range(1, len(pass_rates)):
    improvement = pass_rates[i] - pass_rates[0]
    ax2.annotate(f'+{improvement:.1f}pp',
                xy=(i, pass_rates[i]),
                xytext=(i, pass_rates[i] + 2),
                ha='center',
                fontsize=10,
                color='darkgreen' if improvement > 0 else 'darkred')

plt.tight_layout()
plt.show()

print("\nPerformance Impact Summary:")
print("=" * 60)
print(improvements_df.to_string(index=False))
print(f"\nAverage improvement: {improvements_df['Improvement'].mean():.1f} percentage points")
print(f"Average relative improvement: {improvements_df['Improvement_Pct'].mean():.1f}%")

## 8. Best Practices for Synthetic Data Generation

Based on the paper's success, here are key principles for creating synthetic programming datasets:

In [ ]:
class SyntheticDataGuidelines:
    """Best practices for synthetic programming data generation"""
    
    def __init__(self):
        self.guidelines = {
            "Source Quality": {
                "principle": "Start with high-quality source data",
                "implementation": "Use curated exercise datasets like CodeExercises",
                "impact": "Ensures diverse, educational examples"
            },
            "Translation Fidelity": {
                "principle": "Preserve semantic meaning while adapting idioms",
                "implementation": "Explicit prompt about docstrings and idioms",
                "impact": "Natural, idiomatic target language code"
            },
            "Iterative Validation": {
                "principle": "Monitor quality during translation process",
                "implementation": "Batch translation with quality checks",
                "impact": "Catch and fix systematic issues early"
            },
            "Manual Review": {
                "principle": "Human validation of representative samples",
                "implementation": "Expert review of diverse examples",
                "impact": "Ensures correctness and quality"
            },
            "Diversity Coverage": {
                "principle": "Cover broad spectrum of concepts and difficulties",
                "implementation": "Track concept distribution, ensure balance",
                "impact": "Model learns comprehensive language features"
            },
            "Size Optimization": {
                "principle": "Quality over quantity for synthetic data",
                "implementation": "15K high-quality examples > millions of noisy ones",
                "impact": "Efficient training, better performance"
            }
        }
    
    def create_translation_pipeline(self):
        """Create a complete pipeline following best practices"""
        pipeline_steps = [
            "1. Source Data Selection",
            "   - Choose educational, diverse exercises",
            "   - Ensure concept coverage",
            "",
            "2. Translation Setup",
            "   - Use consistent, explicit prompts",
            "   - Include language-specific requirements",
            "",
            "3. Batch Processing",
            "   - Translate in manageable batches",
            "   - Monitor quality metrics",
            "",
            "4. Quality Control",
            "   - Automated validation checks",
            "   - Flag problematic translations",
            "",
            "5. Manual Review",
            "   - Sample diverse examples",
            "   - Expert validation",
            "",
            "6. Dataset Compilation",
            "   - Filter low-quality translations",
            "   - Balance concept distribution",
            "",
            "7. Testing",
            "   - Train models on subsets",
            "   - Validate performance improvements"
        ]
        
        return pipeline_steps
    
    def display_guidelines(self):
        """Display guidelines in structured format"""
        print("Synthetic Data Generation Best Practices:")
        print("=" * 70)
        
        for category, details in self.guidelines.items():
            print(f"\n{category}:")
            print(f"  Principle: {details['principle']}")
            print(f"  How: {details['implementation']}")
            print(f"  Why: {details['impact']}")

guidelines = SyntheticDataGuidelines()
guidelines.display_guidelines()

print("\n\nTranslation Pipeline:")
print("=" * 50)
for step in guidelines.create_translation_pipeline():
    print(step)

## 9. Future Directions

### 9.1 From Section VII of the Paper

The paper suggests focusing on "generating more synthetic and high-quality code to cover not only coding exercises but also more realistic production tasks."

In [ ]:
# Visualize potential extensions
fig, ax = plt.subplots(figsize=(12, 8))

# Define future directions
future_directions = [
    "Production Code Patterns",
    "Framework-Specific Examples",
    "Error Handling Scenarios",
    "Performance Optimizations",
    "Design Pattern Implementations",
    "Real-World API Integrations",
    "Testing and Debugging Code",
    "Concurrent Programming Examples"
]

current_coverage = [20, 15, 30, 25, 35, 10, 40, 20]  # Simulated current coverage %
potential_impact = [90, 85, 70, 80, 75, 95, 60, 88]  # Simulated potential impact

# Create scatter plot
scatter = ax.scatter(current_coverage, potential_impact, 
                    s=200, alpha=0.6, c=range(len(future_directions)),
                    cmap='viridis')

# Add labels
for i, direction in enumerate(future_directions):
    ax.annotate(direction, (current_coverage[i], potential_impact[i]),
                xytext=(5, 5), textcoords='offset points',
                fontsize=9, alpha=0.8)

# Add quadrant lines
ax.axhline(y=75, color='gray', linestyle='--', alpha=0.5)
ax.axvline(x=30, color='gray', linestyle='--', alpha=0.5)

# Quadrant labels
ax.text(15, 90, 'High Impact\nLow Coverage', ha='center', va='center',
        bbox=dict(boxstyle='round', facecolor='lightgreen', alpha=0.5))
ax.text(45, 90, 'High Impact\nHigh Coverage', ha='center', va='center',
        bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.5))

ax.set_xlabel('Current Coverage in KExercises (%)')
ax.set_ylabel('Potential Impact on Real-World Code Generation (%)')
ax.set_title('Future Directions for Synthetic Kotlin Data Generation')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nKey Opportunities for Expansion:")
print("=" * 50)
high_impact_low_coverage = [
    (direction, impact, coverage) 
    for direction, impact, coverage in zip(future_directions, potential_impact, current_coverage)
    if impact > 75 and coverage < 30
]

for direction, impact, coverage in sorted(high_impact_low_coverage, key=lambda x: x[1], reverse=True):
    print(f"{direction:<30} Impact: {impact}%, Current: {coverage}%")

## 10. Summary and Key Takeaways

### What Made KExercises Successful

1. **High-Quality Source**: Started with curated Python exercises
2. **Simple but Effective Prompt**: Clear, concise translation instruction
3. **Focus on Education**: Exercises designed for learning, not just functionality
4. **Iterative Validation**: Continuous quality monitoring during translation
5. **Right Size**: 15K examples proved more effective than millions of uncurated files

### Key Results

- **Best Performance**: 55.28% pass rate (Deepseek-7B + KExercises)
- **Largest Improvement**: +14.29 percentage points for Deepseek-7B
- **Consistent Gains**: All models improved with KExercises

### Lessons for Other Languages

This approach can be replicated for any low-resource programming language:
1. Find high-quality exercises in a well-resourced language
2. Use LLMs for translation with explicit instructions
3. Validate and curate the results
4. Focus on quality over quantity

In [ ]:
# Final summary visualization
fig, ax = plt.subplots(figsize=(10, 6))

# Data preparation
methods = ['Raw GitHub\n(KStack)', 'Quality Filtered\n(KStack-clean)', 'Synthetic Exercises\n(KExercises)']
improvements = [3.1, 11.8, 16.15]  # Average improvements across models
dataset_sizes = [4000000, 25000, 15000]

# Create bubble chart
colors = ['lightcoral', 'lightblue', 'lightgreen']
for i, (method, improvement, size) in enumerate(zip(methods, improvements, dataset_sizes)):
    # Bubble size proportional to log of dataset size
    bubble_size = np.log10(size) * 100
    ax.scatter(i, improvement, s=bubble_size, c=colors[i], alpha=0.7, edgecolors='black')
    
    # Add size label
    if size >= 1000000:
        size_label = f"{size/1000000:.1f}M"
    else:
        size_label = f"{size/1000:.0f}K"
    ax.text(i, improvement + 1, size_label, ha='center', fontsize=10)

ax.set_xticks(range(len(methods)))
ax.set_xticklabels(methods)
ax.set_ylabel('Average Pass Rate Improvement (pp)')
ax.set_title('Quality vs Quantity: Dataset Impact on Kotlin Code Generation')
ax.grid(True, alpha=0.3, axis='y')
ax.set_ylim(0, 20)

# Add annotation
ax.annotate('Synthetic data wins!\nQuality > Quantity',
            xy=(2, improvements[2]), xytext=(1.5, 18),
            arrowprops=dict(arrowstyle='->', color='green', lw=2),
            fontsize=12, ha='center',
            bbox=dict(boxstyle='round', facecolor='yellow', alpha=0.7))

plt.tight_layout()
plt.show()

print("\nConclusion:")
print("=" * 50)
print("Synthetic data generation through translation is a powerful technique")
print("for improving code generation in low-resource languages.")
print("\nThe key is focusing on quality, diversity, and educational value")
print("rather than simply maximizing dataset size.")